In [1]:
from sagemaker import get_execution_role
role = get_execution_role()

In [2]:
pip install awswrangler

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pygeohash

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import sagemaker
import boto3
import awswrangler as wr

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

import pygeohash as pgh

In [5]:
import os
import boto3

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

import sagemaker
from sagemaker import get_execution_role
import sagemaker_pyspark

role = get_execution_role()

# Configure Spark to use the SageMaker Spark dependency jars
jars = sagemaker_pyspark.classpath_jars()

classpath = ":".join(sagemaker_pyspark.classpath_jars())

# See the SageMaker Spark Github to learn how to connect to EMR from a notebook instance
spark = (
    SparkSession.builder.config("spark.driver.extraClassPath", classpath)\
    .config('spark.driver.memory','32G')\
    .master("local[*]")\
    .getOrCreate()
)

spark

In [6]:
# https://sagemaker-examples.readthedocs.io/en/latest/sagemaker-spark/pyspark_mnist/pyspark_mnist_kmeans.html#Loading-the-Data
import boto3

cn_regions = ["cn-north-1", "cn-northwest-1"]
region = boto3.Session().region_name
endpoint_domain = "com.cn" if region in cn_regions else "com"
spark._jsc.hadoopConfiguration().set(
    "fs.s3a.endpoint", "s3.{}.amazonaws.{}".format(region, endpoint_domain)
)

In [7]:
from pyspark.sql.functions import *

In [8]:
center_name = 'MIAMI' #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< CHOOSE!!!
hour_diff = 5 #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< CHOOSE!!!

time_interval = 6
time_lag = "INTERVAL " + str(hour_diff) + " HOURS"
time_lag

'INTERVAL 5 HOURS'

%%time
latlon = spark.read.format('csv')\
        .options(header= 'true', inferSchema= 'true')\
        .load("s3a://partly-cloudy-common-area/proof_of_concept/latlonRAP.csv")

latlon.count()

latlon.show(5)

geoHashFunc = udf(lambda x,y: pgh.encode(x,y, precision= 5))
latlon = latlon.withColumn("geoEncode", geoHashFunc(col('Lat'), col('Lon')))

latlon.show(5)

latlon = latlon.drop("Lat", "Lon")

%%time
centers = spark.read.format('csv')\
        .options(header= 'true', inferSchema= 'true')\
        .load("s3a://partly-cloudy-common-area/proof_of_concept/rapCellNxNy_ARTCC_intersect.csv")

centers.count()

centers.groupBy(col("NAME")).agg(count(col("NAME"))).show()

centers = centers.drop("IDENT")

%%time
rap = spark.read.format('parquet')\
        .load("s3a://partly-cloudy-rap-parquet/*.parquet")


rap.printSchema()

%%time
rap_centers = rap.join(centers, ['Nx', 'Ny'], "inner")
rap_centers.count()

rap_region = rap_centers.filter(col('NAME') == center_name)
rap_region.show()

%%time
rap_flat200430 = rap_region.filter((col("FLevel") >= 200) & (col("FLevel") <= 430))\
                            .groupBy("dateTime", "Nx", "Ny")\
                            .agg(sum("IsISSR").alias("iSSRs200430"),
                                 mean("Temperature").alias("meanTemp200430"),
                                 mean("RH_ice").alias("meanRH_ice200430"))
rap_flat200430.count()

%%time
rap_flat300370 = rap_region.filter((col("FLevel") >= 300) & (col("FLevel") <= 370))\
                            .groupBy("dateTime", "Nx", "Ny")\
                            .agg(sum("IsISSR").alias("iSSRs300370"),
                                 mean("Temperature").alias("meanTemp300370"),
                                 mean("RH_ice").alias("meanRH_ice300370"))
rap_flat300370.count()

centerName = center_name.lower()
path = "s3a://partly-cloudy-common-area/" + centerName + "_flat300370.parquet"
rap_flat300370.write.parquet(path, mode= "overwrite", compression= 'gzip')

In [9]:
%%time
centerName = center_name.lower()
rap_flat200430 = spark.read.format('parquet')\
                        .load("s3a://partly-cloudy-common-area/" + centerName + "_flat200430.parquet")
rap_flat200430.show()

+-------------------+---+---+-----------+------------------+------------------+
|           dateTime| Nx| Ny|iSSRs200430|    meanTemp200430|  meanRH_ice200430|
+-------------------+---+---+-----------+------------------+------------------+
|2021-05-21 06:00:00|241| 38|          0|         238.25375|21.392500000000002|
|2021-05-21 06:00:00|231| 48|          0| 238.5141666666667|16.036666666666665|
|2021-05-21 06:00:00|232| 48|          0|238.39966666666666|16.608333333333334|
|2021-05-21 06:00:00|236| 49|          0|          237.9935|17.965833333333332|
|2021-05-21 06:00:00|238| 49|          0|237.79533333333333|18.991666666666664|
|2021-05-21 06:00:00|218| 51|          0|239.38391666666666|28.420833333333334|
|2021-05-21 06:00:00|253| 53|          0|235.77466666666666|           40.3075|
|2021-05-20 07:00:00|233| 29|          1|239.32674999999998| 92.94749999999999|
|2021-05-20 07:00:00|248| 58|          0|         236.91525|36.837500000000006|
|2021-05-20 08:00:00|246| 31|          0

In [10]:
%%time
centerName = center_name.lower()
rap_flat300430 = spark.read.format('parquet')\
                        .load("s3a://partly-cloudy-common-area/" + centerName + "_flat300430.parquet")
rap_flat300430.show()

+-------------------+---+---+-----------+------------------+------------------+
|           dateTime| Nx| Ny|iSSRs300430|    meanTemp300430|  meanRH_ice300430|
+-------------------+---+---+-----------+------------------+------------------+
|2021-05-21 06:00:00|241| 38|          0|          224.7545|33.278333333333336|
|2021-05-21 06:00:00|231| 48|          0|225.15016666666668|             22.26|
|2021-05-21 06:00:00|232| 48|          0| 225.0148333333333|23.084999999999997|
|2021-05-21 06:00:00|236| 49|          0|224.65033333333335|26.218333333333334|
|2021-05-21 06:00:00|238| 49|          0|224.47316666666666|27.149999999999995|
|2021-05-21 06:00:00|218| 51|          0|225.91066666666666| 35.64000000000001|
|2021-05-21 06:00:00|253| 53|          0|222.67116666666666|48.178333333333335|
|2021-05-20 07:00:00|233| 29|          1|226.17483333333334| 84.66333333333333|
|2021-05-20 07:00:00|248| 58|          0| 224.2788333333333|            49.095|
|2021-05-20 08:00:00|246| 31|          0

In [11]:
%%time
rap_flat_combined = rap_flat200430.join(rap_flat300430, ['dateTime', 'Nx', 'Ny'], "inner")
rap_flat_combined.show()

+-------------------+---+---+-----------+------------------+------------------+-----------+------------------+------------------+
|           dateTime| Nx| Ny|iSSRs200430|    meanTemp200430|  meanRH_ice200430|iSSRs300430|    meanTemp300430|  meanRH_ice300430|
+-------------------+---+---+-----------+------------------+------------------+-----------+------------------+------------------+
|2020-06-01 00:00:00|224| 29|          0|242.19566666666663| 75.77916666666667|          0|228.70949999999996| 74.90166666666666|
|2020-06-01 00:00:00|242| 41|          1|240.85708333333332| 87.03083333333335|          1|227.49066666666667| 87.41833333333334|
|2020-06-01 00:00:00|249| 56|          0|239.20091666666664|21.029999999999998|          0| 225.7303333333333| 35.60333333333333|
|2020-06-01 01:00:00|220| 44|          0| 241.2899166666667| 69.89666666666666|          0|227.96783333333335|63.153333333333336|
|2020-06-01 01:00:00|228| 33|          0|241.74816666666666|            82.795|          0

%%time
rap_flat_combined_pdPandas = rap_flat_combined.toPandas()
rap_flat_combined_pdPandas

%%time
rap_flat200430 = rap_flat200430.withColumn("localDateTime", col("dateTime") - expr(time_lag))
rap_flat200430.count()
rap_flat200430 = rap_flat200430.withColumn("dateOnly", to_date(col("localDateTime")))
rap_flat200430 = rap_flat200430.withColumn("hour", hour(col("localDateTime")))
periodFunc = udf(lambda x: x//time_interval * time_interval)
rap_flat200430 = rap_flat200430.withColumn("period", periodFunc(col('hour')))
rap_flat200430.show()

%%time
rap_flat300370 = rap_flat_combined.withColumn("localDateTime", col("dateTime") - expr(time_lag))
rap_flat300370.count()
rap_flat300370 = rap_flat300370.withColumn("dateOnly", to_date(col("localDateTime")))
rap_flat300370 = rap_flat300370.withColumn("hour", hour(col("localDateTime")))
periodFunc = udf(lambda x: x//time_interval * time_interval)
rap_flat300370 = rap_flat300370.withColumn("period", periodFunc(col('hour')))
rap_flat300370.show()

In [12]:
%%time
rap_flat_combined = rap_flat_combined.withColumn("localDateTime", col("dateTime") - expr(time_lag))
rap_flat_combined.count()
rap_flat_combined = rap_flat_combined.withColumn("dateOnly", to_date(col("localDateTime")))
rap_flat_combined = rap_flat_combined.withColumn("hour", hour(col("localDateTime")))
periodFunc = udf(lambda x: x//time_interval * time_interval)
rap_flat_combined = rap_flat_combined.withColumn("period", periodFunc(col('hour')))
rap_flat_combined.count()

CPU times: user 6.07 ms, sys: 5.68 ms, total: 11.8 ms
Wall time: 16.8 s


7142700

In [13]:
rap_flat_combined.show(5)

+-------------------+---+---+-----------+------------------+------------------+-----------+------------------+------------------+-------------------+----------+----+------+
|           dateTime| Nx| Ny|iSSRs200430|    meanTemp200430|  meanRH_ice200430|iSSRs300430|    meanTemp300430|  meanRH_ice300430|      localDateTime|  dateOnly|hour|period|
+-------------------+---+---+-----------+------------------+------------------+-----------+------------------+------------------+-------------------+----------+----+------+
|2020-06-01 00:00:00|224| 29|          0|242.19566666666663| 75.77916666666667|          0|228.70949999999996| 74.90166666666666|2020-05-31 19:00:00|2020-05-31|  19|    18|
|2020-06-01 00:00:00|242| 41|          1|240.85708333333332| 87.03083333333335|          1|227.49066666666667| 87.41833333333334|2020-05-31 19:00:00|2020-05-31|  19|    18|
|2020-06-01 00:00:00|249| 56|          0|239.20091666666664|21.029999999999998|          0| 225.7303333333333| 35.60333333333333|2020-0

centerName = center_name.lower()
path = "s3a://partly-cloudy-common-area/" + centerName + "_rap_flat_combined.parquet"
rap_flat_combined.write.parquet(path, mode= "overwrite", compression= 'gzip')

centerName = center_name.lower()
path = "s3a://partly-cloudy-ml-inputs-parquet/" + centerName + "_flat200430.parquet"
rap_flat200430.write.parquet(path, mode= "overwrite", compression= 'gzip')

In [14]:
rap_flat_combined_pd = rap_flat_combined.filter((col('hour') == 0) | (col('hour') == 6) | (col('hour') == 12) | (col('hour') == 18))
rap_flat_combined_pd.count()

1190450

%%time
rap_flat_combined_pd = rap_flat_combined.groupBy("dateOnly", "period", "Nx", "Ny")\
                                        .agg(sum("iSSRs200430").alias("pdISSRs200430"),
                                             sum("iSSRs300370").alias("pdISSRs300370"),
                                             mean("meanTemp200430").alias("pdMeanTemp200430"),
                                             mean("meanTemp300370").alias("pdMeanTemp300370"),
                                             mean("meanRH_ice200430").alias("pdMeanRH_ice200430"),
                                             mean("meanRH_ice300370").alias("pdMeanRH_ice300370")
                                            )

rap_flat_combined_pd.count()

%%time
rap_flat_combined_pd = rap_flat300370.groupBy(col("dateOnly"), col("period"), col("Nx"), col("Ny"))\
                                        .agg(#sum("iSSRs200430").alias("pdISSRs200430"),
                                            sum("iSSRs300370").alias("pdISSRs300370"),
                                            #mean("meanTemp200430").alias("pdMeanTemp200430"),
                                            mean("meanTemp300370").alias("pdMeanTemp300370"),
                                            #mean("meanRH_ice200430").alias("pdMeanRH_ice200430"),
                                            mean("meanRH_ice300370").alias("pdMeanRH_ice300370")
                                            )

rap_flat_combined_pd.count()

centerName = center_name.lower()
path = "s3a://partly-cloudy-common-area/" + centerName + "_6_hour_flat300370.parquet"
rap_flat_combined_pd.write.parquet(path, mode= "overwrite", compression= 'gzip')

%%time
centerName = center_name.lower()
rap_flat_combined_pd = spark.read.format('parquet')\
                            .load("s3a://partly-cloudy-common-area/" + centerName + "_6_hour_flat.parquet")


In [15]:
%%time
rap_flat_combined_pdPandas = rap_flat_combined_pd.toPandas()
rap_flat_combined_pdPandas

CPU times: user 1min 8s, sys: 393 ms, total: 1min 8s
Wall time: 1min 17s


,dateTime,Nx,Ny,iSSRs200430,meanTemp200430,meanRH_ice200430,iSSRs300430,meanTemp300430,meanRH_ice300430,localDateTime,dateOnly,hour,period
0,2020-06-01 05:00:00,233,33,0,241.380000,81.680833,0,228.127833,73.265000,2020-06-01 00:00:00,2020-06-01,0,0
1,2020-06-01 05:00:00,242,37,0,240.911250,71.379167,0,227.492500,64.171667,2020-06-01 00:00:00,2020-06-01,0,0
2,2020-06-01 11:00:00,221,37,0,241.250417,80.720000,0,227.910167,75.161667,2020-06-01 06:00:00,2020-06-01,6,6
3,2020-06-01 11:00:00,221,47,2,240.859917,82.760833,2,227.660333,83.446667,2020-06-01 06:00:00,2020-06-01,6,6
4,2020-06-01 11:00:00,230,45,0,240.297417,70.797500,0,227.170667,80.940000,2020-06-01 06:00:00,2020-06-01,6,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1190445,2021-05-31 11:00:00,226,39,0,236.294833,14.505833,0,223.526333,20.475000,2021-05-31 06:00:00,2021-05-31,6,6
1190446,2021-05-31 11:00:00,237,43,0,235.857333,21.837500,0,223.620167,18.385000,2021-05-31 06:00:00,2021-05-31,6,6
1190447,2021-05-31 11:00:00,238,38,0,236.305417,21.536667,0,224.172333,19.570000,2021-05-31 06:00:00,2021-05-31,6,6
1190448,2021-05-31 11:00:00,242,36,0,236.680250,48.703333,0,224.411667,39.025000,2021-05-31 06:00:00,2021-05-31,6,6


In [16]:
%%time
rap_flat_combined_pdPandas['LocalDateTimePeriod'] = [pd.Timestamp(rap_flat_combined_pdPandas['dateOnly'][i]) + timedelta(hours= int(rap_flat_combined_pdPandas['period'][i])) for i in range(len(rap_flat_combined_pdPandas))]
rap_flat_combined_pdPandas

CPU times: user 28.7 s, sys: 0 ns, total: 28.7 s
Wall time: 28.7 s


,dateTime,Nx,Ny,iSSRs200430,meanTemp200430,meanRH_ice200430,iSSRs300430,meanTemp300430,meanRH_ice300430,localDateTime,dateOnly,hour,period,LocalDateTimePeriod
0,2020-06-01 05:00:00,233,33,0,241.380000,81.680833,0,228.127833,73.265000,2020-06-01 00:00:00,2020-06-01,0,0,2020-06-01 00:00:00
1,2020-06-01 05:00:00,242,37,0,240.911250,71.379167,0,227.492500,64.171667,2020-06-01 00:00:00,2020-06-01,0,0,2020-06-01 00:00:00
2,2020-06-01 11:00:00,221,37,0,241.250417,80.720000,0,227.910167,75.161667,2020-06-01 06:00:00,2020-06-01,6,6,2020-06-01 06:00:00
3,2020-06-01 11:00:00,221,47,2,240.859917,82.760833,2,227.660333,83.446667,2020-06-01 06:00:00,2020-06-01,6,6,2020-06-01 06:00:00
4,2020-06-01 11:00:00,230,45,0,240.297417,70.797500,0,227.170667,80.940000,2020-06-01 06:00:00,2020-06-01,6,6,2020-06-01 06:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1190445,2021-05-31 11:00:00,226,39,0,236.294833,14.505833,0,223.526333,20.475000,2021-05-31 06:00:00,2021-05-31,6,6,2021-05-31 06:00:00
1190446,2021-05-31 11:00:00,237,43,0,235.857333,21.837500,0,223.620167,18.385000,2021-05-31 06:00:00,2021-05-31,6,6,2021-05-31 06:00:00
1190447,2021-05-31 11:00:00,238,38,0,236.305417,21.536667,0,224.172333,19.570000,2021-05-31 06:00:00,2021-05-31,6,6,2021-05-31 06:00:00
1190448,2021-05-31 11:00:00,242,36,0,236.680250,48.703333,0,224.411667,39.025000,2021-05-31 06:00:00,2021-05-31,6,6,2021-05-31 06:00:00


In [17]:
%%time
rap_flat_combined_pdPandas.drop(columns= ['dateOnly', 'period'], inplace= True)

CPU times: user 66.3 ms, sys: 28.9 ms, total: 95.1 ms
Wall time: 94 ms


In [18]:
%%time
centers_df = wr.s3.read_csv('s3://partly-cloudy-common-area/proof_of_concept/rapCellNxNy_ARTCC_intersect.csv')
centers_df

CPU times: user 116 ms, sys: 8.28 ms, total: 124 ms
Wall time: 269 ms


,IDENT,NAME,Nx,Ny
0,ZMA,MIAMI,233,28
1,ZMA,MIAMI,242,29
2,ZMA,MIAMI,232,28
3,ZMA,MIAMI,241,29
4,ZMA,MIAMI,249,30
...,...,...,...,...
25318,ZSE,SEATTLE,76,173
25319,ZLC,SALT LAKE CITY,114,168
25320,ZLC,SALT LAKE CITY,126,167
25321,ZMP,MINNEAPOLIS,144,166


In [19]:
%%time
latlon_df = wr.s3.read_csv('s3://partly-cloudy-common-area/proof_of_concept/latlonRAP.csv')
latlon_df

CPU times: user 137 ms, sys: 12.1 ms, total: 150 ms
Wall time: 384 ms


,Nx,Ny,Lat,Lon
0,1,1,16.281000,-126.138000
1,2,1,16.322011,-125.954684
2,3,1,16.362789,-125.771252
3,4,1,16.403332,-125.587705
4,5,1,16.443642,-125.404045
...,...,...,...,...
67720,297,225,55.648911,-58.431595
67721,298,225,55.607604,-58.167947
67722,299,225,55.565986,-57.904583
67723,300,225,55.524058,-57.641507


In [20]:
%%time
center_flat_pdPandas_join = pd.merge(rap_flat_combined_pdPandas, centers_df, on= ['Nx', 'Ny'])
center_flat_pdPandas_join = pd.merge(center_flat_pdPandas_join, latlon_df, on= ['Nx', 'Ny'])
center_flat_pdPandas_join

CPU times: user 401 ms, sys: 96.2 ms, total: 497 ms
Wall time: 496 ms


,dateTime,Nx,Ny,iSSRs200430,meanTemp200430,meanRH_ice200430,iSSRs300430,meanTemp300430,meanRH_ice300430,localDateTime,hour,LocalDateTimePeriod,IDENT,NAME,Lat,Lon
0,2020-06-01 05:00:00,233,33,0,241.380000,81.680833,0,228.127833,73.265000,2020-06-01 00:00:00,0,2020-06-01 00:00:00,ZMA,MIAMI,24.921705,-81.276090
1,2020-10-04 23:00:00,233,33,1,243.788250,33.925833,1,230.275333,58.361667,2020-10-04 18:00:00,18,2020-10-04 18:00:00,ZMA,MIAMI,24.921705,-81.276090
2,2020-10-23 23:00:00,233,33,0,241.995833,49.405833,0,228.875167,52.705000,2020-10-23 18:00:00,18,2020-10-23 18:00:00,ZMA,MIAMI,24.921705,-81.276090
3,2020-11-05 17:00:00,233,33,0,239.876000,22.170833,0,227.803333,39.806667,2020-11-05 12:00:00,12,2020-11-05 12:00:00,ZMA,MIAMI,24.921705,-81.276090
4,2020-12-12 17:00:00,233,33,0,238.642750,45.038333,0,225.028667,66.601667,2020-12-12 12:00:00,12,2020-12-12 12:00:00,ZMA,MIAMI,24.921705,-81.276090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1190445,2020-11-11 11:00:00,248,62,0,241.433833,91.675000,0,228.475333,90.830000,2020-11-11 06:00:00,6,2020-11-11 06:00:00,ZMA,MIAMI,29.865399,-77.527569
1190446,2020-12-29 17:00:00,248,62,0,235.397167,25.524167,0,225.834500,10.753333,2020-12-29 12:00:00,12,2020-12-29 12:00:00,ZMA,MIAMI,29.865399,-77.527569
1190447,2021-01-28 05:00:00,248,62,0,236.545250,51.263333,0,223.002167,67.741667,2021-01-28 00:00:00,0,2021-01-28 00:00:00,ZMA,MIAMI,29.865399,-77.527569
1190448,2021-04-14 05:00:00,248,62,0,235.330167,38.975833,0,224.255333,51.000000,2021-04-14 00:00:00,0,2021-04-14 00:00:00,ZMA,MIAMI,29.865399,-77.527569


In [21]:
%%time
center_flat_pdPandas_join['geoEncode'] = [pgh.encode(center_flat_pdPandas_join['Lat'][i], center_flat_pdPandas_join['Lon'][i], precision= 5) for i in range(len(center_flat_pdPandas_join))]
center_flat_pdPandas_join

CPU times: user 52.6 s, sys: 50.2 ms, total: 52.7 s
Wall time: 52.7 s


,dateTime,Nx,Ny,iSSRs200430,meanTemp200430,meanRH_ice200430,iSSRs300430,meanTemp300430,meanRH_ice300430,localDateTime,hour,LocalDateTimePeriod,IDENT,NAME,Lat,Lon,geoEncode
0,2020-06-01 05:00:00,233,33,0,241.380000,81.680833,0,228.127833,73.265000,2020-06-01 00:00:00,0,2020-06-01 00:00:00,ZMA,MIAMI,24.921705,-81.276090,dhqjy
1,2020-10-04 23:00:00,233,33,1,243.788250,33.925833,1,230.275333,58.361667,2020-10-04 18:00:00,18,2020-10-04 18:00:00,ZMA,MIAMI,24.921705,-81.276090,dhqjy
2,2020-10-23 23:00:00,233,33,0,241.995833,49.405833,0,228.875167,52.705000,2020-10-23 18:00:00,18,2020-10-23 18:00:00,ZMA,MIAMI,24.921705,-81.276090,dhqjy
3,2020-11-05 17:00:00,233,33,0,239.876000,22.170833,0,227.803333,39.806667,2020-11-05 12:00:00,12,2020-11-05 12:00:00,ZMA,MIAMI,24.921705,-81.276090,dhqjy
4,2020-12-12 17:00:00,233,33,0,238.642750,45.038333,0,225.028667,66.601667,2020-12-12 12:00:00,12,2020-12-12 12:00:00,ZMA,MIAMI,24.921705,-81.276090,dhqjy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1190445,2020-11-11 11:00:00,248,62,0,241.433833,91.675000,0,228.475333,90.830000,2020-11-11 06:00:00,6,2020-11-11 06:00:00,ZMA,MIAMI,29.865399,-77.527569,dm2cg
1190446,2020-12-29 17:00:00,248,62,0,235.397167,25.524167,0,225.834500,10.753333,2020-12-29 12:00:00,12,2020-12-29 12:00:00,ZMA,MIAMI,29.865399,-77.527569,dm2cg
1190447,2021-01-28 05:00:00,248,62,0,236.545250,51.263333,0,223.002167,67.741667,2021-01-28 00:00:00,0,2021-01-28 00:00:00,ZMA,MIAMI,29.865399,-77.527569,dm2cg
1190448,2021-04-14 05:00:00,248,62,0,235.330167,38.975833,0,224.255333,51.000000,2021-04-14 00:00:00,0,2021-04-14 00:00:00,ZMA,MIAMI,29.865399,-77.527569,dm2cg


In [22]:
%%time
center_flat_pd_final = center_flat_pdPandas_join[['localDateTime', 
                                                  'Nx', 'Ny', 
                                                  'Lat', 'Lon', 
                                                  'geoEncode', 
                                                  'NAME', 
                                                  'iSSRs200430', 
                                                  'iSSRs300430',
                                                  'meanTemp200430',
                                                  'meanTemp300430',
                                                  'meanRH_ice200430',
                                                  'meanRH_ice300430']].copy()
center_flat_pd_final

CPU times: user 180 ms, sys: 36 ms, total: 216 ms
Wall time: 215 ms


,localDateTime,Nx,Ny,Lat,Lon,geoEncode,NAME,iSSRs200430,iSSRs300430,meanTemp200430,meanTemp300430,meanRH_ice200430,meanRH_ice300430
0,2020-06-01 00:00:00,233,33,24.921705,-81.276090,dhqjy,MIAMI,0,0,241.380000,228.127833,81.680833,73.265000
1,2020-10-04 18:00:00,233,33,24.921705,-81.276090,dhqjy,MIAMI,1,1,243.788250,230.275333,33.925833,58.361667
2,2020-10-23 18:00:00,233,33,24.921705,-81.276090,dhqjy,MIAMI,0,0,241.995833,228.875167,49.405833,52.705000
3,2020-11-05 12:00:00,233,33,24.921705,-81.276090,dhqjy,MIAMI,0,0,239.876000,227.803333,22.170833,39.806667
4,2020-12-12 12:00:00,233,33,24.921705,-81.276090,dhqjy,MIAMI,0,0,238.642750,225.028667,45.038333,66.601667
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1190445,2020-11-11 06:00:00,248,62,29.865399,-77.527569,dm2cg,MIAMI,0,0,241.433833,228.475333,91.675000,90.830000
1190446,2020-12-29 12:00:00,248,62,29.865399,-77.527569,dm2cg,MIAMI,0,0,235.397167,225.834500,25.524167,10.753333
1190447,2021-01-28 00:00:00,248,62,29.865399,-77.527569,dm2cg,MIAMI,0,0,236.545250,223.002167,51.263333,67.741667
1190448,2021-04-14 00:00:00,248,62,29.865399,-77.527569,dm2cg,MIAMI,0,0,235.330167,224.255333,38.975833,51.000000


In [23]:
output_bucket = 'partly-cloudy-ml-inputs'
outputFileName = center_name + '_every_6_hour_flat.csv' 

In [24]:
wr.s3.to_csv(center_flat_pd_final, f"s3://{output_bucket}/{outputFileName}", index=False)

{'paths': ['s3://partly-cloudy-ml-inputs/MIAMI_every_6_hour_flat.csv'],
 'partitions_values': {}}